In [21]:
# Import all necessary libraries
import pandas as pd
print('Pandas version: ', pd.__version__)
import numpy as np
print('NumPy version: ', np.__version__)
import pickle
print('Pickle version: ', pickle.format_version)
from sklearn.feature_extraction.text import CountVectorizer

Pandas version:  1.5.0
NumPy version:  1.23.4
Pickle version:  4.0


In [13]:
carts = pd.read_csv('all_carts_encoded.csv')
tickets = np.unique(carts['TICKET_ID'])[:10000]

In [31]:
model = pickle.load(open(f'models/kmeans/0.pkl', 'rb'))
clusters = model.labels_

In [23]:
tickets_items_list = pickle.load(open('items_list_for_ticket.txt', 'rb'))
def prepSparseMatrix(list_of_str):
    cv = CountVectorizer(token_pattern = r'[^\,\ ]+', lowercase = False)
    sparseMatrix = cv.fit_transform(list_of_str)
    return sparseMatrix, cv.get_feature_names_out()
sparseMatrix, feature_names = prepSparseMatrix(tickets_items_list[:10000])
df_sparseMatrix = pd.DataFrame.sparse.from_spmatrix(sparseMatrix, index = tickets, columns = feature_names)

In [11]:
# Display each ticket and its corresponding cluster (need clusters given my fit_transform a model)
tickets_cluster = pd.DataFrame(np.concatenate((tickets.reshape(-1,1), clusters.reshape(-1,1)), axis = 1), columns = ['TICKET_ID', 'Cluster'])
tickets_cluster.T

,0,1,2,3,4,5,6,7,8,9,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
TICKET_ID,32931447,32931448,32931451,32931452,32931453,32931454,32931455,32931456,32931461,32931462,...,32945415,32945421,32945423,32945424,32945426,32945430,32945431,32945432,32945434,32945435
Cluster,0,0,0,0,0,0,0,0,0,3,...,1,0,0,0,2,0,3,0,0,0


In [14]:
# For each cluster, how many times do we find each libelle
def clustersLibelles(tickets_cluster, tickets_data):
    clusters = list(tickets_cluster['Cluster'])
    each_cluster_libelles = list()
    for i in range(len(np.unique(clusters))):
        tickets_list = list(tickets_cluster[tickets_cluster['Cluster'] == i]['TICKET_ID'])
        tickets_libelles_list = list()
        for ticket in tickets_list:    
            tickets_libelles_list.extend(list(tickets_data[tickets_data['TICKET_ID'] == ticket]['LIBELLE_ENCODED']))
        tickets_libelles_counts = list()
        tickets_libelles_counts.extend([[libelle, tickets_libelles_list.count(libelle)] for libelle in np.unique(tickets_libelles_list)])
        each_cluster_libelles.append(pd.DataFrame(tickets_libelles_counts, columns=['LIBELLE_ENCODED', 'Count']).sort_values(by = ['Count'], ascending = False).reset_index(drop=True))
    return each_cluster_libelles

cluster_libelles = clustersLibelles(tickets_cluster, carts)

In [15]:
# For cluster 1, libelle [LIBELLE_ENCODED] is present [Count] times
cluster_libelles[1].T

,0,1,2,3,4,5,6,7,8,9,...,550,551,552,553,554,555,556,557,558,559
LIBELLE_ENCODED,302,1082,990,1334,336,1305,214,941,57,357,...,746,656,744,710,708,686,674,672,659,1477
Count,683,44,23,19,18,17,15,13,13,13,...,1,1,1,1,1,1,1,1,1,1


In [16]:
# How many different tickets are in each cluster ?
# Also check that the sum of the nb of tickets in each cluster == the total number of tickets at start
total_nb_tickets = 0
for i in range(8):
    len_tickets = tickets_cluster[tickets_cluster['Cluster'] == i].shape[0]
    total_nb_tickets += len_tickets
    print('Tickets in Cluster ' + str(i) + ' -> ', len_tickets) 
print(f"Total number of tickets : {total_nb_tickets}")

Tickets in Cluster 0 ->  7874
Tickets in Cluster 1 ->  683
Tickets in Cluster 2 ->  388
Tickets in Cluster 3 ->  750
Tickets in Cluster 4 ->  33
Tickets in Cluster 5 ->  62
Tickets in Cluster 6 ->  41
Tickets in Cluster 7 ->  169
Total number of tickets : 10000


In [17]:
# Get all the tickets belonging to a certain client (by his id)
tickets_of_clients = pd.read_csv('KaDo.csv', usecols = ['TICKET_ID', 'CLI_ID'])
def tickets_of_a_client(client_id):
    tickets_df = tickets_of_clients[tickets_of_clients['CLI_ID'] == client_id]
    return tickets_df['TICKET_ID'].unique()    
    
analyzed_client_tickets = tickets_of_a_client(941958669)
analyzed_client_tickets

array([32946317, 33182830, 33577174, 34015044, 34398198, 34665272,
       35017400, 35221068, 35546515, 35703330, 35816290, 35998180,
       36061723, 36315860, 36434434])

In [29]:
# Get all the clusters of a specific client thanks to his former tickets
def clusters_of_a_client(tickets):
    client_clusters = list()
    for ticket_id in tickets:
        current_cluster = tickets_cluster[tickets_cluster['TICKET_ID'] == ticket_id]['Cluster'].values[0]
        if current_cluster not in client_clusters:
            client_clusters.append(current_cluster)
    return client_clusters

analyzed_client_clusters = clusters_of_a_client(analyzed_client_tickets)
analyzed_client_clusters

[0]

In [25]:
def bought_items(tickets):
    bought_items = list()
    for ticket_id in tickets:
        libelles_serie = df_sparseMatrix.loc[ticket_id]
        for item_id in list(libelles_serie[libelles_serie==1].index):
            bought_items.append(item_id)
    return list(map(int,bought_items))
            
analyzed_client_bought_items = bought_items(analyzed_client_tickets)
analyzed_client_bought_items

KeyError: 32946317

In [26]:
def items_contained_in_clusters(clusters):
    items_contained_in_clusters = list()
    for cluster in clusters:
        items = cluster_libelles[cluster][cluster_libelles[cluster]['Count'] > 500]['LIBELLE_ENCODED'].values
        for item in items:
            if item not in items_contained_in_clusters:
                items_contained_in_clusters.append(item)
    return items_contained_in_clusters

items_in_clusters = items_contained_in_clusters(analyzed_client_clusters)
items_in_clusters

NameError: name 'analyzed_client_clusters' is not defined

In [27]:
def items_suggestion(bought_items, items_in_clusters):
    suggested_items = list()
    for item in items_in_clusters:
        if item not in bought_items:
            suggested_items.append(item)
    return suggested_items

suggested_items = items_suggestion(analyzed_client_bought_items, items_in_clusters)
suggested_items

NameError: name 'analyzed_client_bought_items' is not defined

In [ ]:
# Decode labels so we can read the
decoded_suggested_items.append(le.inverse_transform(si))
decoded_suggested_items